<a href="https://colab.research.google.com/github/vi-sync/aai_ccollab/blob/main/lstm_donor_model_lastfail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!gdown --id 1GpATd_pM4mcnWWIs28-s1lgqdAg2Wdv-
!gdown --id 1pGd5tLwA30M7wkbJKdXHaae9tYVDICJ_
!gdown --id 16oSeZfQjX5LfF1KFOq98pNjVzR0YNzYb
!gdown --id 17qamrBz5YdIL62O256qvRwra3BfIJryL

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1GpATd_pM4mcnWWIs28-s1lgqdAg2Wdv-
To: /content/preprocessed_data.csv
100% 124M/124M [00:00<00:00, 375MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1pGd5tLwA30M7wkbJKdXHaae9tYVDICJ_
To: /content/glove_vectors
100% 128M/128M [00:00<00:00, 286MB/s] 
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
Fr

In [2]:
import pandas as pd
import pickle as pk
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing import text
from tensorflow.keras import utils
from tensorflow.keras.layers import Conv1D

from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import LSTM,Embedding,Flatten,Dense,concatenate,Dropout,BatchNormalization
from tensorflow.keras.regularizers import l1,l2
from keras import Model,Input
from tensorflow.keras.utils import plot_model
from tensorflow.keras import callbacks


In [3]:

preprocessed_data=pd.read_csv("preprocessed_data.csv")
preprocessed_data

,school_state,teacher_prefix,project_grade_category,teacher_number_of_previously_posted_projects,project_is_approved,clean_categories,clean_subcategories,essay,price
0,ca,mrs,grades_prek_2,53,1,math_science,appliedsciences health_lifescience,i fortunate enough use fairy tale stem kits cl...,725.05
1,ut,ms,grades_3_5,4,1,specialneeds,specialneeds,imagine 8 9 years old you third grade classroo...,213.03
2,ca,mrs,grades_prek_2,10,1,literacy_language,literacy,having class 24 students comes diverse learner...,329.00
3,ga,mrs,grades_prek_2,2,1,appliedlearning,earlydevelopment,i recently read article giving students choice...,481.04
4,wa,mrs,grades_3_5,2,1,literacy_language,literacy,my students crave challenge eat obstacles brea...,17.74
...,...,...,...,...,...,...,...,...,...
109243,hi,mrs,grades_9_12,5,1,health_sports,teamsports,our day starts 100 students athletes low incom...,143.36
109244,nm,ms,grades_prek_2,3,1,appliedlearning,earlydevelopment other,my students range age four five years old atte...,268.57
109245,il,mrs,grades_3_5,0,1,math_science,appliedsciences environmentalscience,we title 1 school 650 total students our eleme...,399.00
109246,hi,mrs,grades_9_12,1,1,math_science,health_lifescience,i teach many different types students my class...,287.73


In [4]:
res_data=pd.read_csv("/content/resources_lstm_donor.csv")
res_data

,id,description,quantity,price
0,p233245,LC652 - Lakeshore Double-Space Mobile Drying Rack,1,149.00
1,p069063,Bouncy Bands for Desks (Blue support pipes),3,14.95
2,p069063,Cory Stories: A Kid's Book About Living With Adhd,1,8.45
3,p069063,"Dixon Ticonderoga Wood-Cased #2 HB Pencils, Bo...",2,13.59
4,p069063,EDUCATIONAL INSIGHTS FLUORESCENT LIGHT FILTERS...,3,24.95
...,...,...,...,...
1541267,p031981,AmazonBasics 9 Volt Everyday Alkaline Batterie...,1,9.99
1541268,p031981,AmazonBasics AAA Performance Alkaline Batterie...,1,6.99
1541269,p031981,Black Electrical Tape (GIANT 3 PACK) Each Roll...,6,8.99
1541270,p031981,Flormoon DC Motor Mini Electric Motor 0.5-3V 1...,2,8.14


In [5]:
train_data=pd.read_csv('/content/train_data_lstm_donor.csv')
train_data

,Unnamed: 0,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,project_is_approved
0,160221,p253737,c90749f5d961ff158d4b4d1e7dc665fc,Mrs.,IN,2016-12-05 13:43:57,Grades PreK-2,Literacy & Language,"ESL, Literacy",Educational Support for English Learners at Home,My students are English learners that are work...,"\""The limits of your language are the limits o...",NaN,NaN,My students need opportunities to practice beg...,0,0
1,140945,p258326,897464ce9ddc600bced1151f324dd63a,Mr.,FL,2016-10-25 09:22:10,Grades 6-8,"History & Civics, Health & Sports","Civics & Government, Team Sports",Wanted: Projector for Hungry Learners,Our students arrive to our school eager to lea...,The projector we need for our school is very c...,NaN,NaN,My students need a projector to help with view...,7,1
2,21895,p182444,3465aaf82da834c0582ebd0ef8040ca0,Ms.,AZ,2016-08-31 12:03:56,Grades 6-8,Health & Sports,"Health & Wellness, Team Sports",Soccer Equipment for AWESOME Middle School Stu...,"\r\n\""True champions aren't always the ones th...",The students on the campus come to school know...,NaN,NaN,"My students need shine guards, athletic socks,...",1,0
3,45,p246581,f3cb9bffbba169bef1a77b243e620b60,Mrs.,KY,2016-10-06 21:16:17,Grades PreK-2,"Literacy & Language, Math & Science","Literacy, Mathematics",Techie Kindergarteners,I work at a unique school filled with both ESL...,My students live in high poverty conditions wi...,NaN,NaN,My students need to engage in Reading and Math...,4,1
4,172407,p104768,be1f7507a41f8479dc06f047086a39ec,Mrs.,TX,2016-07-11 01:10:09,Grades PreK-2,Math & Science,Mathematics,Interactive Math Tools,Our second grade classroom next year will be m...,"For many students, math is a subject that does...",NaN,NaN,My students need hands on practice in mathemat...,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109243,38267,p048540,fadf72d6cd83ce6074f9be78a6fcd374,Mr.,MO,2016-06-17 12:02:31,Grades PreK-2,"Literacy & Language, Math & Science","Literature & Writing, Mathematics",Privacy Shields Help Promote Independent Thinking,Welcome to Mr. Ramos's 2nd grade classroom! We...,I would like to start preparing my second grad...,NaN,NaN,My students need these privacy partitions to h...,0,1
109244,169142,p166281,1984d915cc8b91aa16b4d1e6e39296c6,Ms.,NJ,2017-01-11 12:49:39,Grades PreK-2,"Literacy & Language, Math & Science","Literacy, Mathematics",Technology in Our Classroom,"Every morning, we start our day with our core ...","In this technological age, we need to give our...",NaN,NaN,My students need two iPad's and protective cas...,0,1
109245,143653,p155633,cdbfd04aa041dc6739e9e576b1fb1478,Mrs.,NJ,2016-08-25 17:11:32,Grades PreK-2,"Literacy & Language, Math & Science","Literacy, Mathematics",2016/2017 Beginning of the Year Basics,This is a great group of sharing and caring st...,"My students learn about special events, holida...",NaN,NaN,My students need giant comfy pillows in order ...,3,1
109246,164599,p206114,6d5675dbfafa1371f0e2f6f1b716fe2d,Mrs.,NY,2016-07-29 17:53:15,Grades 3-5,"Health & Sports, Special Needs","Health & Wellness, Special Needs",Flexible Seating in Inclusive Classroom,Our students live in a small rural community. ...,Flexible classroom seating has been researched...,NaN,NaN,My students need flexible seating options: bea...,0,1


# adding price,quantity and proj_res_summary_contains_num to preprocesed data

In [6]:
preprocessed_data.columns

Index(['school_state', 'teacher_prefix', 'project_grade_category',
       'teacher_number_of_previously_posted_projects', 'project_is_approved',
       'clean_categories', 'clean_subcategories', 'essay', 'price'],
      dtype='object')

In [7]:
def check_num(line):
  return any(ele.isdigit() for ele in line)

In [8]:
preprocessed_data['proj_res_summary_contains_num']=train_data['project_resource_summary'].apply(lambda x : check_num(x))

In [9]:
preprocessed_data['proj_res_summary_contains_num']=preprocessed_data['proj_res_summary_contains_num'].apply(lambda x : 1 if True else 0)

In [10]:
preprocessed_data

,school_state,teacher_prefix,project_grade_category,teacher_number_of_previously_posted_projects,project_is_approved,clean_categories,clean_subcategories,essay,price,proj_res_summary_contains_num
0,ca,mrs,grades_prek_2,53,1,math_science,appliedsciences health_lifescience,i fortunate enough use fairy tale stem kits cl...,725.05,1
1,ut,ms,grades_3_5,4,1,specialneeds,specialneeds,imagine 8 9 years old you third grade classroo...,213.03,1
2,ca,mrs,grades_prek_2,10,1,literacy_language,literacy,having class 24 students comes diverse learner...,329.00,1
3,ga,mrs,grades_prek_2,2,1,appliedlearning,earlydevelopment,i recently read article giving students choice...,481.04,1
4,wa,mrs,grades_3_5,2,1,literacy_language,literacy,my students crave challenge eat obstacles brea...,17.74,1
...,...,...,...,...,...,...,...,...,...,...
109243,hi,mrs,grades_9_12,5,1,health_sports,teamsports,our day starts 100 students athletes low incom...,143.36,1
109244,nm,ms,grades_prek_2,3,1,appliedlearning,earlydevelopment other,my students range age four five years old atte...,268.57,1
109245,il,mrs,grades_3_5,0,1,math_science,appliedsciences environmentalscience,we title 1 school 650 total students our eleme...,399.00,1
109246,hi,mrs,grades_9_12,1,1,math_science,health_lifescience,i teach many different types students my class...,287.73,1


***Couldnt add quantity because of the non matching number of records ***

In [11]:
x_df=preprocessed_data.drop(columns=['project_is_approved'])
x_df

,school_state,teacher_prefix,project_grade_category,teacher_number_of_previously_posted_projects,clean_categories,clean_subcategories,essay,price,proj_res_summary_contains_num
0,ca,mrs,grades_prek_2,53,math_science,appliedsciences health_lifescience,i fortunate enough use fairy tale stem kits cl...,725.05,1
1,ut,ms,grades_3_5,4,specialneeds,specialneeds,imagine 8 9 years old you third grade classroo...,213.03,1
2,ca,mrs,grades_prek_2,10,literacy_language,literacy,having class 24 students comes diverse learner...,329.00,1
3,ga,mrs,grades_prek_2,2,appliedlearning,earlydevelopment,i recently read article giving students choice...,481.04,1
4,wa,mrs,grades_3_5,2,literacy_language,literacy,my students crave challenge eat obstacles brea...,17.74,1
...,...,...,...,...,...,...,...,...,...
109243,hi,mrs,grades_9_12,5,health_sports,teamsports,our day starts 100 students athletes low incom...,143.36,1
109244,nm,ms,grades_prek_2,3,appliedlearning,earlydevelopment other,my students range age four five years old atte...,268.57,1
109245,il,mrs,grades_3_5,0,math_science,appliedsciences environmentalscience,we title 1 school 650 total students our eleme...,399.00,1
109246,hi,mrs,grades_9_12,1,math_science,health_lifescience,i teach many different types students my class...,287.73,1


In [12]:
y_df=preprocessed_data['project_is_approved'].values
y_df

array([1, 1, 1, ..., 1, 1, 1])

In [13]:
preprocessed_data[preprocessed_data.isnull().any(axis=1)]

,school_state,teacher_prefix,project_grade_category,teacher_number_of_previously_posted_projects,project_is_approved,clean_categories,clean_subcategories,essay,price,proj_res_summary_contains_num


# splitting

In [14]:
x_train,x_test,y_train,y_test=train_test_split(x_df,y_df,stratify=y_df,train_size=0.8)
print(x_train.shape,x_test.shape)
print(y_train.shape,y_test.shape)

(87398, 9) (21850, 9)
(87398,) (21850,)


In [15]:
# categorising y
y_train = utils.to_categorical(y_train, 2)
y_test = utils.to_categorical(y_test, 2)

# Tokenizing ESSAY using keras

In [16]:
tokenizer=text.Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(x_train['essay'])
vocab_size=len(tokenizer.word_index)+1
x_train_seq=tokenizer.texts_to_sequences(x_train['essay'])
x_test_seq=tokenizer.texts_to_sequences(x_test['essay'])  
print(len(tokenizer.word_counts))
print(len(tokenizer.word_index))


51517
51518


In [17]:
# Padding essay
max_len_essay=max([len(x.split()) for x in x_train['essay']])
print(max_len_essay)
x_train_pad_essay=pad_sequences(x_train_seq,maxlen=max_len_essay)
x_test_pad_essay=pad_sequences(x_test_seq,maxlen=max_len_essay)



339


# loading the glove pk file 300 dim

In [18]:
with open("/content/glove_vectors",'rb') as f:
  glove_vectors=pk.load(f)

embedding_matrix=np.zeros((vocab_size,300))

for word,index in tokenizer.word_index.items():
  if word in glove_vectors:
    embedding_matrix[index]=glove_vectors.get(word)

print('Number of words not foung in the GlOVE file',len(np.where(~embedding_matrix.any(axis=1))[0]))



Number of words not foung in the GlOVE file 5572


# *** CATEGORICAL FEATURES***

## 1 SCHOOL STATE

In [19]:
tokenizer_school=text.Tokenizer(oov_token="<OOV>")
tokenizer_school.fit_on_texts(x_train['school_state'])
vocab_size_school=len(tokenizer_school.word_index)+1
print('vocab size is ',vocab_size_school)

#text to seq
x_train_seq_school=tokenizer_school.texts_to_sequences(x_train['school_state'])
x_test_seq_school=tokenizer_school.texts_to_sequences(x_test['school_state'])

#padding
max_len_school=max([len(x.split(" ")) for x in x_train['school_state']])
print('max len is ',max_len_school)
x_train_pad_school=pad_sequences(x_train_seq_school,maxlen=max_len_school)
x_test_pad_school=pad_sequences(x_test_seq_school,maxlen=max_len_school)

#embedding
#https://mmuratarat.github.io/2019-06-12/embeddings-with-numeric-variables-Keras#:~:text=Embedding%20Dimensionality&text=Jeremy%20Howard%20provides%20a%20general,So%20it's%20kind%20of%20experimental.
number_of_categories=x_train.school_state.nunique()
embedding_size_school= int(min(50, np.ceil(number_of_categories/2)))
print("vec space size for school sttaes is" ,embedding_size_school)

vocab size is  53
max len is  1
vec space size for school sttaes is 26


# 3  PROJECT GRADE CAT

In [20]:
tokenizer_grad_cat=text.Tokenizer(oov_token="<OOV>")
tokenizer_grad_cat.fit_on_texts(x_train['project_grade_category'])
#tokenizer_grad_cat.word_counts
vocab_size_grad_cat=len(tokenizer_grad_cat.word_index)+1
print("vocab_size_grad_cat",vocab_size_grad_cat)

x_train_seq_grad_cat=tokenizer_grad_cat.texts_to_sequences(x_train['project_grade_category'])
x_tets_seq_grad_cat=tokenizer_grad_cat.texts_to_sequences(x_test['project_grade_category'])

#padding
#if used split("_") then we could have get max len 3 
max_len_proj_grad=max([len(x.split(" ")) for x in x_train['project_grade_category']])
print('max len proj grade',max_len_proj_grad)


x_train_pad_proj_grad=pad_sequences(x_train_seq_grad_cat,max_len_proj_grad)
x_test_pad_proj_grad=pad_sequences(x_tets_seq_grad_cat,max_len_proj_grad)
#pad_x_tr_proj_grad

#embedding
embedding_size_proj_grad=int(min(np.ceil(x_train['project_grade_category'].nunique()/2),50))
print("vector sp[ace size of proj grad cat is ",embedding_size_proj_grad)


vocab_size_grad_cat 11
max len proj grade 1
vector sp[ace size of proj grad cat is  2


# 4 # TEACHER PREFIX

In [21]:
#tokenizer_teacher_pre=text.Tokenizer(oov_token="<OOV>")
tokenizer_teacher_pre=text.Tokenizer()
tokenizer_teacher_pre.fit_on_texts(x_train['teacher_prefix'])
vocab_size_teacer_pre=len(tokenizer_teacher_pre.word_index)+1
print('vocab size is ',vocab_size_teacer_pre)

#text tom seq
x_train_seq_teacher_pre=tokenizer_teacher_pre.texts_to_sequences(x_train['teacher_prefix'])
x_test_seq_teacher_pre=tokenizer_teacher_pre.texts_to_sequences(x_test['teacher_prefix'])

max_len_teacher_pre=max([len(x.split()) for x in x_train['teacher_prefix']])
print("max_len_teacher_pre",max_len_teacher_pre)

#padingh
x_train_pad_teacher_pre=pad_sequences(sequences=x_train_seq_teacher_pre,maxlen=max_len_teacher_pre)
x_test_pad_teacher_pre=pad_sequences(sequences=x_test_seq_teacher_pre,maxlen=max_len_teacher_pre)
#embedding
embedding_size_teacher_pre = int(min(np.ceil(x_train['teacher_prefix'].nunique()/2),50))
print("embedding_size_teacher_pre",embedding_size_teacher_pre)

vocab size is  6
max_len_teacher_pre 1
embedding_size_teacher_pre 3


In [22]:
tokenizer_teacher_pre.word_index

{'dr': 5, 'mr': 3, 'mrs': 1, 'ms': 2, 'teacher': 4}

In [23]:
preprocessed_data.teacher_prefix.nunique()

5

In [24]:
preprocessed_data.teacher_prefix.value_counts()

mrs        57272
ms         38955
mr         10648
teacher     2360
dr            13
Name: teacher_prefix, dtype: int64

In [25]:
x_train.teacher_prefix.value_counts()

mrs        45758
ms         31242
mr          8510
teacher     1880
dr             8
Name: teacher_prefix, dtype: int64

# 5 CLEAN CAT

In [26]:
tokenizer_clean_cat=text.Tokenizer(oov_token="OOV")
tokenizer_clean_cat.fit_on_texts(x_train['clean_categories'])

vocab_size_clean_cat=len(tokenizer_clean_cat.word_index)+1
print('vocab size is ',vocab_size_clean_cat)
x_train_seq_clean_cat=tokenizer_clean_cat.texts_to_sequences(x_train['clean_categories'])
x_test_seq_clean_cat=tokenizer_clean_cat.texts_to_sequences(x_test['clean_categories'])

# padding
max_len_clean_cat=max([len(x.split()) for x in x_train['clean_categories']])
print('max len is ',max_len_clean_cat)
x_train_pad_clean_cat=pad_sequences(x_train_seq_clean_cat,maxlen=max_len_clean_cat)
x_test_pad_clean_cat=pad_sequences(x_test_seq_clean_cat,maxlen=max_len_clean_cat)

# embedding size 
embedding_size_clean_cat=int(min(np.ceil(x_train['clean_categories'].nunique()/2),50))
print("embedding_size_clean_cat",embedding_size_clean_cat)

vocab size is  17
max len is  3
embedding_size_clean_cat 26


# 6  CLEAN SUB CAT

In [27]:
tokenizer_clean_subcat=text.Tokenizer(oov_token="<OOV>")
tokenizer_clean_subcat.fit_on_texts(x_train['clean_subcategories'])
vocab_size_clean_subcat =   len(tokenizer_clean_subcat.word_index)+1
print("vocab_size_clean_subcat",vocab_size_clean_subcat)

x_train_seq_clean_subcat=tokenizer_clean_subcat.texts_to_sequences(x_train['clean_subcategories'])
x_test_seq_clean_subcat=tokenizer_clean_subcat.texts_to_sequences(x_test['clean_subcategories'])

# padiing
 
max_len_clean_subcat=max([len(x.split()) for x in x_train['clean_subcategories']])
print("max_len_clean_subcat",max_len_clean_subcat)

x_train_pad_clean_subcat=pad_sequences(x_train_seq_clean_subcat,maxlen=max_len_clean_cat)
x_test_pad_clean_subcat=pad_sequences(x_test_seq_clean_subcat,maxlen=max_len_clean_cat)

# embedding size
embedding_size_clean_subcat=min(int(np.ceil(x_train.clean_subcategories.nunique()/2)),50)
print("embedding_size_clean_subcat",embedding_size_clean_subcat)

vocab_size_clean_subcat 39
max_len_clean_subcat 3
embedding_size_clean_subcat 50


# 7  NUMERICAL DATA
# 7.1  proj prev posted


In [28]:
standardscaler=StandardScaler()

In [29]:
x_train_num_prevproj= standardscaler.fit_transform(x_train['teacher_number_of_previously_posted_projects'].values.reshape(-1,1))
x_test_num_prevproj=standardscaler.transform(x_test.teacher_number_of_previously_posted_projects.values.reshape(-1,1))

# 7.2 PRICE

In [30]:
x_train_price=standardscaler.fit_transform(x_train['price'].values.reshape(-1,1))
x_test_price=standardscaler.transform(x_test['price'].values.reshape(-1,1))

# 7.3 resource_summary_contains_numerical_digits

In [31]:
x_train_res_sum_num=standardscaler.fit_transform(x_train['proj_res_summary_contains_num'].values.reshape(-1,1))
x_test_res_sum_num=standardscaler.transform(x_test['proj_res_summary_contains_num'].values.reshape(-1,1))

In [32]:
4 5 1 2  f 

SyntaxError: ignored

# # CONCATINATING num features 

# ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

In [ ]:
4 5 d s d 2

In [33]:

x_train_num_features=np.concatenate((x_train_num_prevproj,x_train_price,x_train_res_sum_num),axis=1)
x_test_num_features=np.concatenate((x_test_num_prevproj,x_test_price,x_test_res_sum_num),axis=1)

In [34]:
print(x_train_num_features.shape ,x_test_num_features.shape)


(87398, 3) (21850, 3)


#***MODEL 1***

## ***CREATING CALLBACKS*** 

In [35]:
from sklearn.metrics import roc_auc_score
import tensorflow as tf
def auroc(y_true, y_pred):
  return tf.numpy_function(roc_auc_score, (y_true, y_pred), tf.double)



In [36]:
import os
from datetime import datetime

#tensoboard
name="MODEL_1"
logdir=os.path.join('logs/',datetime.now().strftime("%Y-%m-%d/%H-%M-%S"),name)
tf_cb=callbacks.TensorBoard(log_dir=logdir,histogram_freq=1)

#learning rate platue
lr_platue=callbacks.ReduceLROnPlateau()

"""def lr_decay(epoch,lr):
    if epoch!=0 and epoch%3==0:
      lr=lr*0.5
    return lr
lr_schedular=callbacks.LearningRateScheduler(lr_decay)
"""
#saving weights
path='model_checkpont/'
checkpoint=callbacks.ModelCheckpoint(filepath=path,save_weights_only=True)


#callbackslist=[tf_cb,lr_platue,lr_schedular,checkpoint]
callbackslist=[tf_cb,lr_platue,checkpoint]

### Defining inputs

In [ ]:
#input layer total text,essay

input_essay=Input((max_len_essay,),dtype='int32',name="input_essay")
print(input_essay.shape)
embedding_layer_essay= Embedding(vocab_size,output_dim=300,weights=[embedding_matrix],trainable=False )(input_essay)
lstm_essay= LSTM(128,return_sequences=True, activation = 'tanh',recurrent_dropout=0.5)(embedding_layer_essay)

flat_essay=Flatten()(lstm_essay)


In [ ]:
#input layer school state

input_ss=Input((max_len_school,),dtype='int32',name="input_ss")
print(input_ss.shape)
embedding_layer_schoolstate=Embedding(vocab_size_school,output_dim=embedding_size_school)(input_ss)
flat_schoolstate=Flatten()(embedding_layer_schoolstate)

In [ ]:
#input layer proj grade category

input_grade=Input((max_len_proj_grad,),dtype='int32',name="input_grade")
print(input_grade.shape)
embedding_layer_proj_grad=Embedding(input_dim=vocab_size_grad_cat,output_dim=embedding_size_proj_grad)(input_grade)
flat_proj_grad=Flatten()(embedding_layer_proj_grad)

In [ ]:
#input_ layer clean categories

input_cc=Input((max_len_clean_cat,),dtype='int32',name="input_cc")
print(input_cc.shape)
embedding_layer_clean_cat=Embedding(input_dim=vocab_size_clean_cat,output_dim=embedding_size_clean_cat)(input_cc)
flat_clean_cat=Flatten()(embedding_layer_clean_cat)

In [ ]:
#input layer clean_sub categories

input_subcat=Input((max_len_clean_subcat,),dtype='int32',name='input_subcat')
print(input_subcat.shape)
embedding_layer_clean_subcat=Embedding(input_dim=vocab_size_clean_subcat,output_dim=embedding_size_clean_subcat)(input_subcat)
flat_layer_clean_sucat=Flatten()(embedding_layer_clean_subcat)

In [ ]:
#input layer teacger prefix

input_tp=Input(shape=(max_len_teacher_pre,),dtype='int32',name="input_tp")
print(input_tp.shape)
embedding_layer_teacher_pre=Embedding(input_dim=vocab_size_teacer_pre,output_dim=embedding_size_teacher_pre)(input_tp)
flat_layer_teacher_pre=Flatten()(embedding_layer_teacher_pre)

In [ ]:
#numerical features

numeric_features = Input((3,),dtype='int32',name='numeric_features')
print(numeric_features.shape)
numeric_dense = Dense(32, activation='relu' , kernel_initializer='he_normal',kernel_regularizer=l2(1e-4))(numeric_features)

### MODEL Architecture 

In [ ]:
#   CONACATINATING LAYERS
concatted=tf.keras.layers.concatenate([flat_essay,flat_schoolstate,flat_layer_teacher_pre,\
                                       flat_clean_cat,flat_layer_clean_sucat,flat_proj_grad,numeric_dense])

#concatted= concatenate([flat_essay,flat_schoolstate,flat_proj_grad,flat_clean_cat,flat_layer_clean_sucat,flat_layer_teacher_pre,numeric_dense])
dense1 = Dense(512, activation="relu", kernel_initializer="he_normal" ,kernel_regularizer=l2(0.001))(concatted)

do1 = Dropout(0.5)(dense1)

dense2 = Dense(256,activation="relu",kernel_initializer="he_normal" ,kernel_regularizer=l2(0.001))(do1)

bn1 = BatchNormalization()(dense2)

do2 = Dropout(0.5)(bn1)

dense3 = Dense(128,activation="relu", kernel_initializer="he_normal" ,kernel_regularizer=l2(0.001))(do2)


output = Dense(2, activation='softmax', name='output')(dense3)

model1= Model(inputs=[input_essay, input_ss ,input_tp,input_cc,
                       input_subcat ,input_grade ,numeric_features ],outputs=[output])

# training inputs 
train_x=[x_train_pad_essay ,x_train_pad_school ,x_train_pad_teacher_pre, x_train_pad_clean_cat , x_train_pad_clean_subcat,x_train_pad_proj_grad ,  x_train_num_features]
test_x=[x_test_pad_essay   ,x_test_pad_school  ,x_test_pad_teacher_pre,  x_test_pad_clean_cat  , x_test_pad_clean_subcat, x_test_pad_proj_grad  ,  x_test_num_features]

# -----------------------------------------------------------===================================================================================

In [ ]:
--------------------------------------------------

In [ ]:
model1.summary()

In [ ]:
#compiling
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[auroc])
history=model1.fit(
    x=train_x,
    y=y_train,
    batch_size=512,
    epochs=7,
    verbose=1,
    callbacks=callbackslist,
    validation_data=(test_x,y_test),
    validation_split=0.30
)


In [ ]:
%reload_ext tensorboard
%tensorboard --logdir logs


In [ ]:
utils.plot_model(model1,to_file='model1.png')

In [ ]:
history.history['val_auroc']

In [ ]:
result=model1.evaluate(test_x,y_test,batch_size=512)

In [ ]:
print("Val auc model 1 is ",result[1]*100," %")

## **Val auc model 1 is  75.97509622573853  %**

# **MODEL 2**

In [ ]:
import shutil
shutil.rmtree("/content/logs_2", ignore_errors=True)
shutil.rmtree("/content/model2_checkpoint/checkpoint",ignore_errors=True)

In [ ]:
import os
from datetime import datetime

#tensoboard
name="MODEL_2"
logdir=os.path.join('logs_2/',datetime.now().strftime("%Y-%m-%d/%H-%M-%S"),name)
tf_cb=callbacks.TensorBoard(log_dir=logdir,histogram_freq=1)

#learning rate platue
lr_platue=callbacks.ReduceLROnPlateau()

"""def lr_decay(epoch,lr):
    if epoch!=0 and epoch%3==0:
      lr=lr*0.5
    return lr
lr_schedular=callbacks.LearningRateScheduler(lr_decay)
"""
#saving weights
path='model2_checkpoint/'
checkpoint=callbacks.ModelCheckpoint(filepath=path,save_weights_only=True,monitor="val_auroc",mode='max',verbose=1)



#callbackslist=[tf_cb,lr_platue,lr_schedular,checkpoint]
callbackslist=[tf_cb,lr_platue,checkpoint]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
import nltk
nltk.download('stopwords')
stopwords=set(nltk.corpus.stopwords.words('english'))

In [ ]:
vectorizer=TfidfVectorizer(max_features=10000,min_df=5,lowercase=True,stop_words=stopwords)
x_tr_essay_tfidf=vectorizer.fit_transform(x_train['essay'])
idf_dict=dict(zip(vectorizer.get_feature_names(),vectorizer.idf_))

idf_df=pd.DataFrame({'word':idf_dict.keys(),'values':idf_dict.values()})
idf_df

In [ ]:
print('word with highets idf value = ',max(idf_dict,key=lambda x: idf_dict[x]))
print('word with lowest idf value = ',min(idf_dict,key=lambda x: idf_dict[x]))

In [ ]:
sns.boxplot(data=idf_df)

In [ ]:
sns.displot(vectorizer.idf_, kde=True)
plt.show()

In [ ]:
idf_df['values'].quantile([0.05, 0.25, 0.5,0.75, 0.90,1.0])

#### choosing 25 to 90 percentile

In [ ]:
best_idf=idf_df[(idf_df['values']>2) & (idf_df['values']<10)]
best_idf_word=best_idf['word'].to_list()
print('No of choosen words is',len(best_idf_word))

#converting list of best idf values to the set
set_bestidf_words=set(best_idf_word)



##  *removing non best idf words from train and test set *

In [ ]:
def remove_low_high_idf(line,set_bestidf_words):
  l=[]
  for word in line.split():
    if word in set_bestidf_words:
      l.append(word)
  return ' '.join(l)

In [ ]:
x_train['essay_idf']=x_train['essay'].apply(lambda x: remove_low_high_idf(x,set_bestidf_words))
x_test['essay_idf']=x_test['essay'].apply(lambda x: remove_low_high_idf(x,set_bestidf_words))

In [ ]:
"""
for line in a:
  l=[]
  for word in line.split():
    if word in set_bestidf_words:
       l.append(word)
  print(' '.join(l))
  print("-----------------------------")
     

"""

## tokenizing on basis of new data

In [ ]:
tokenizer_idf=text.Tokenizer(oov_token='<OOV>')
tokenizer_idf.fit_on_texts(x_train['essay_idf'])
#vocab
print("vocab size is" , len(tokenizer_idf.word_index)+1)
vocab_size_essay_idf= len(tokenizer_idf.word_index)+1
#max(tokenizer.word_index,key=lambda x: tokenizer.word_index[x])

tr_max_len_essay_idf=max([len(line.split())  for line in x_train['essay_idf'].to_list()])
te_max_len_essay_idf=max([len(line.split())  for line in x_test['essay_idf'].to_list()])

print('train ',tr_max_len_essay_idf)
print('test ',te_max_len_essay_idf)

if tr_max_len_essay_idf < te_max_len_essay_idf:
  max_len_essayidf = te_max_len_essay_idf
  print('since test set has the longest sentence therefore max_len= ',te_max_len_essay_idf)
else:
  max_len_essayidf = tr_max_len_essay_idf
  print('since train set has the longest sentence therefore max_len= ',tr_max_len_essay_idf)


#print('maximun lenth of trai and train amd test data are ',tr_max_len_essay_idf,' and ',te_max_len_essay_idf,' respectively We willchoose the max out these two')
#print('max leght of padding is ',te_max_len_essay_idf)
#max_len_essayidf=te_max_len_essay_idf

#text to seq
tr_seq_essay=tokenizer_idf.texts_to_sequences(x_train['essay_idf'])
te_seq_essay=tokenizer_idf.texts_to_sequences(x_test['essay_idf'])

# padding
tr_essayidf_pad=pad_sequences(tr_seq_essay,maxlen=max_len_essayidf)
te_essayidf_pad=pad_sequences(te_seq_essay,maxlen=max_len_essayidf)


In [ ]:
#embedding
embedding_matrix_idf=np.zeros(shape=(vocab_size_essay_idf,300))
with open("/content/glove_vectors",'rb') as f:
  glove_vectors=pk.load(f)

for word,index in tokenizer_idf.word_index.items():
  if word in glove_vectors:
    embedding_matrix_idf[index]=glove_vectors.get(word)

In [ ]:
#input layer essay_idf

input_essay_idf=Input((max_len_essayidf,),dtype='int32',name="input_essay_idf")
print(input_essay_idf.shape)
embedding_layer_essay_idf= Embedding(vocab_size_essay_idf,output_dim=300,weights=[embedding_matrix_idf],trainable=False )(input_essay_idf)
lstm_essay_idf= LSTM(128,return_sequences=True, activation = 'tanh',recurrent_dropout=0.5)(embedding_layer_essay_idf)

flat_essay_idf=Flatten()(lstm_essay_idf)


In [ ]:
#   CONACATINATING LAYERS
concatted=tf.keras.layers.concatenate([flat_essay_idf,flat_schoolstate,flat_layer_teacher_pre,\
                                       flat_clean_cat,flat_layer_clean_sucat,flat_proj_grad,numeric_dense])

#concatted= concatenate([flat_essay,flat_schoolstate,flat_proj_grad,flat_clean_cat,flat_layer_clean_sucat,flat_layer_teacher_pre,numeric_dense])
dense1 = Dense(512, activation="relu", kernel_initializer="he_normal" ,kernel_regularizer=l2(0.001))(concatted)

do1 = Dropout(0.5)(dense1)

dense2 = Dense(256,activation="relu",kernel_initializer="he_normal" ,kernel_regularizer=l2(0.001))(do1)

bn1 = BatchNormalization()(dense2)

do2 = Dropout(0.5)(bn1)

dense3 = Dense(128,activation="relu", kernel_initializer="he_normal" ,kernel_regularizer=l2(0.001))(do2)


output = Dense(2, activation='softmax', name='output')(dense3)

model2= Model(inputs=[input_essay_idf, input_ss ,input_tp,input_cc,
                       input_subcat ,input_grade ,numeric_features ],outputs=[output])

# training inputs 
train_x=[tr_essayidf_pad ,x_train_pad_school ,x_train_pad_teacher_pre, x_train_pad_clean_cat , x_train_pad_clean_subcat,x_train_pad_proj_grad ,  x_train_num_features]
test_x=[te_essayidf_pad   ,x_test_pad_school  ,x_test_pad_teacher_pre,  x_test_pad_clean_cat  , x_test_pad_clean_subcat, x_test_pad_proj_grad  ,  x_test_num_features]

In [ ]:
model2.summary()

In [ ]:
model2.compile('adam','categorical_crossentropy',metrics=[auroc])

In [ ]:

history2=model2.fit(
    x=train_x,
    y=y_train,
    batch_size=512,
    epochs=7,
    verbose=1,
    callbacks=callbackslist,
    validation_data=(test_x,y_test),
    validation_split=0.30
)


In [ ]:


%reload_ext tensorboard
%tensorboard --logdir logs_2


In [ ]:
result=model2.evaluate(test_x,y_test,batch_size=512)
print("Val auc model 1 is ",result[1]*100," %")

## MODEL 2 Val auc model 1 is  75.3291130065918  %

In [ ]:
utils.plot_model(model2,'model_2.png')

In [ ]:
4 t4 t 

# ----------------------------------------------------

# 0000000000000000000000000000000000002222222222222222222dddddddddddd        kkkkkkkkkkkk

# 22222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222@@@@@@@@@@@@@@@@@@@@@@

# MODEL 3

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:

from sklearn.metrics import roc_auc_score
import tensorflow as tf
def auroc(y_true, y_pred):
  return tf.numpy_function(roc_auc_score, (y_true, y_pred), tf.double)


import shutil
shutil.rmtree("/content/logs_3", ignore_errors=True)

import os
from datetime import datetime

#tensoboard
name="MODEL_3"
logdir=os.path.join('logs_3/',datetime.now().strftime("%Y-%m-%d/%H-%M-%S"),name)
tf_cb=callbacks.TensorBoard(log_dir=logdir,histogram_freq=1)

#learning rate platue
lr_platue=callbacks.ReduceLROnPlateau()

"""def lr_decay(epoch,lr):
    if epoch!=0 and epoch%3==0:
      lr=lr*0.5
    return lr
lr_schedular=callbacks.LearningRateScheduler(lr_decay)
"""
#saving weights
path='model3_checkpoint/'
checkpoint=callbacks.ModelCheckpoint(filepath=path,save_weights_only=True,monitor='val_auroc',verbose=1,mode='max')


#callbackslist=[tf_cb,lr_platue,lr_schedular,checkpoint]
callbackslist=[tf_cb,lr_platue,checkpoint]

In [ ]:
#school state
encoder=OneHotEncoder(handle_unknown='ignore')
encoder.fit(x_train['school_state'].values.reshape(-1,1))
print('categories are',encoder.categories_[0])
x_tr_ss_ohe=encoder.transform(x_train['school_state'].values.reshape(-1,1))
print(x_tr_ss_ohe.shape)
x_te_ss_ohe=encoder.transform(x_test['school_state'].values.reshape(-1,1))



In [ ]:
#tecgher prefix
encoder=OneHotEncoder(handle_unknown='ignore')
encoder.fit(x_train['teacher_prefix'].values.reshape(-1,1))
print('categories are',encoder.categories_[0])
x_tr_tp_ohe=encoder.transform(x_train['teacher_prefix'].values.reshape(-1,1))
print(x_tr_tp_ohe.shape)
x_te_tp_ohe=encoder.transform(x_test['teacher_prefix'].values.reshape(-1,1))


In [ ]:
#clean categories
encoder=OneHotEncoder(handle_unknown='ignore')
encoder.fit(x_train['clean_categories'].values.reshape(-1,1))
print('categories are',encoder.categories_[0])
x_tr_cc_ohe=encoder.transform(x_train['clean_categories'].values.reshape(-1,1))
print(x_tr_cc_ohe.shape)
x_te_cc_ohe=encoder.transform(x_test['clean_categories'].values.reshape(-1,1))


In [ ]:
#sub categories
encoder=OneHotEncoder(handle_unknown='ignore')
encoder.fit(x_train['clean_subcategories'].values.reshape(-1,1))
print('categories are',encoder.categories_[0])
x_tr_subcat_ohe=encoder.transform(x_train['clean_subcategories'].values.reshape(-1,1))
print(x_tr_subcat_ohe.shape)
x_te_subcat_ohe=encoder.transform(x_test['clean_subcategories'].values.reshape(-1,1))

In [ ]:
#grade categories
encoder=OneHotEncoder(handle_unknown='ignore')
encoder.fit(x_train['project_grade_category'].values.reshape(-1,1))
print('categories are',encoder.categories_[0])
x_tr_grade_ohe=encoder.transform(x_train['project_grade_category'].values.reshape(-1,1))
print(x_tr_grade_ohe.shape)
x_te_grade_ohe=encoder.transform(x_test['project_grade_category'].values.reshape(-1,1))

In [ ]:
## CONCATINATING ALL THE OHE MAtrices
from scipy.sparse import hstack
x_tr_ohe=hstack((x_tr_ss_ohe,x_tr_tp_ohe,x_tr_cc_ohe,x_tr_subcat_ohe, x_tr_grade_ohe,x_train_num_features))
x_te_ohe=hstack((x_te_ss_ohe,x_te_tp_ohe,x_te_cc_ohe,x_te_subcat_ohe, x_te_grade_ohe,x_test_num_features))
print(x_tr_ohe.shape,x_te_ohe.shape)

#converting todemse
x_tr_dense=x_tr_ohe.todense()
x_te_dense=x_te_ohe.todense()

#reshaping for CNN!D 
x_tr_cnn1=np.resize(x_tr_dense,new_shape=(87398,508,1))     #these are all the categorical plus numerical features
x_te_cnn1=np.resize(x_te_dense,new_shape=(21850,508,1))
print('shapes after eshaping ')
print(x_tr_cnn1.shape,x_te_cnn1.shape)              


In [ ]:
# model arch 

#input layer total text,essay

input_essay=Input((max_len_essay,),dtype='int32',name="input_essay")
print(input_essay.shape)
embedding_layer_essay= Embedding(vocab_size,output_dim=300,weights=[embedding_matrix],trainable=False )(input_essay)
lstm_essay= LSTM(128,return_sequences=True, activation = 'tanh',recurrent_dropout=0.5)(embedding_layer_essay)

flat_essay=Flatten()(lstm_essay)

#CONV 1D OTHER THAN ESSAY
input_categorical_num=Input(shape=(508,1),name='input_categorical_num')
conv1d_1 = Conv1D(filters=128,kernel_size=3, strides=1)(input_categorical_num)
print(conv1d_1.shape)
conv1d_2 = Conv1D(filters=64,kernel_size=3, strides=1)(conv1d_1)
print(conv1d_2.shape)
flat_CNN = Flatten()(conv1d_2)


#concating 
concat_cnn=tf.keras.layers.concatenate([flat_essay,flat_CNN])

dense_cnn1=Dense(512,'relu',kernel_initializer='he_normal',kernel_regularizer=l2(0.001))(concat_cnn)
do1=Dropout(0.5)(dense_cnn1)
dense_cnn2=Dense(256,kernel_initializer='he_normal',kernel_regularizer=l2(0.001))(do1)
do2=Dropout(0.5)(dense_cnn2)
output=Dense(2,'softmax')(do2)

model_3=Model(inputs=[input_essay,input_categorical_num],outputs=[output])

model_3.summary()


In [ ]:
train_x_cnn=[x_train_pad_essay,x_tr_cnn1]
test_x_cnn=[x_test_pad_essay,x_te_cnn1]

In [ ]:
utils.plot_model(model_3)

In [ ]:
model_3.compile('adam','categorical_crossentropy',metrics=[auroc])
history_3=model_3.fit(x=train_x_cnn,y=y_train,callbacks=callbackslist,\
                       batch_size=128, epochs=10, verbose=1,\
                      validation_data=(test_x_cnn,y_test),validation_split=0.30)
#crashed with batcg_size=512